In [13]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import psycopg2
from config import db_password

In [15]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [25]:
# Load the data
# file_path = Path('Resources/Airbnb_Fall_Listings.csv.gz')

# def clean_df(file_path):

#     # read csv to dataframe

#     full_df = pd.read_csv(file_path, compression='gzip')

#     # select columns we're interested in for ml model and dashboard

#     analysis_df = full_df[["price","neighbourhood_cleansed","room_type","accommodates","longitude","latitude","beds","bedrooms",

#         "property_type","bathrooms_text","review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", 

#         "review_scores_checkin", "review_scores_communication", "review_scores_location", "review_scores_value", "license", "host_name"]]

#     # process NaNs

#     # drop nan beds and bathrooms_text

#     analysis_df.dropna(subset = ['beds', 'bathrooms_text'], inplace = True)


    # replace NaNs with values based on columns.  Average ratings for review_scores to 2 dec places,

    # text for host_name and license

    # values = {"host_name":"No Host Listed", 

    #           "review_scores_rating": round(analysis_df['review_scores_rating'].mean(), 2), 

    #           "review_scores_accuracy": round(analysis_df['review_scores_accuracy'].mean(), 2),

    #           "review_scores_cleanliness": round(analysis_df['review_scores_cleanliness'].mean(), 2),

    #           "review_scores_checkin": round(analysis_df['review_scores_checkin'].mean(), 2),

    #           "review_scores_communication": round(analysis_df['review_scores_communication'].mean(), 2),

    #           "review_scores_location": round(analysis_df['review_scores_location'].mean(), 2),

    #           "review_scores_value": round(analysis_df['review_scores_value'].mean(), 2),

    #           "bedrooms": analysis_df.loc[full_df['bedrooms'].isna()]['beds'],

    #           "license": "No License"}

    # analysis_df = analysis_df.fillna(value = values)


    # # convert price to float

    # analysis_df['price'] = analysis_df['price'].str.replace('$', '')

#     analysis_df['price'] = analysis_df['price'].str.replace(',', '').astype(float)


#     # convert bathrooms_text to float

#     analysis_df['bathrooms_text'] = analysis_df['bathrooms_text'].str.split(n=1, expand=True)[0]

#     # all bathroom_text values without a number are half baths, verified with value_counts() function

#     # all text values contain the letter a, other values are purely numeric at this point

#     analysis_df.loc[analysis_df['bathrooms_text'].str.contains('a'), 'bathrooms_text'] = 0.5


#     # rename bathrooms_text as bathrooms

#     analysis_df['bathrooms'] = analysis_df['bathrooms_text'].astype(float)

#     analysis_df.drop(['bathrooms_text'], axis = 1, inplace = True)


#     return analysis_df 


# cleaned_df=clean_df(file_path)
# cleaned_df



In [31]:
# Documentation: https://naysan.ca/2020/05/31/postgresql-to-pandas/#:~:text=PostgreSQL%20to%20Pandas%201%20Step%201%3
# A%20Connect%20to,conn.cursor%28%29%20try%3A%20cursor.execute%28select_query%29%20...%203%20Step%203%3A%20Demo

# Documentation 2: https://skytowner.com/explore/comprehensive_guide_on_importing_tables_from_postgresql_as_pandas_dataframes

param_dic = {
    "host"      : "localhost",
    "database"  : "AirBNB_Data",
    "user"      : "postgres",
    "password"  : db_password
}
def connect(param_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**param_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

connecting = connect(param_dic)
connecting

Connecting to the PostgreSQL database...
Connection successful


<connection object at 0x00000241EB28EA68; dsn: 'user=postgres password=xxx dbname=AirBNB_Data host=localhost', closed: 0>

In [35]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df


In [50]:
conn = connect(param_dic)

column_names = ["index", "type", "neighbourhood_clensed", ""]

Df = postgresql_to_dataframe(conn,"select * from airbnb _fall_ml", column_names)
Df

Connecting to the PostgreSQL database...
Connection successful


ValueError: 1 columns passed, passed data had 20 columns

# Split the Data into Training and Testing

In [ ]:
# Create our features
analysis_df = df.drop("", axis=1)

X = pd.get_dummies()
X.head()
#Drop Pymnt_plan column
#Drop Tax_liens column

# # Create our target
y = df[[]]


In [1]:
# X.describe()

In [3]:
# Check the balance of our target values
# y[].value_counts()

In [5]:
from sklearn.model_selection import train_test_split
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

NameError: name 'X' is not defined

### Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
brfc = BalancedRandomForestClassifier(random_state=1, n_estimators=50).fit(X_train_scaled, y_train)
print(f'Training Score: {brfc.score(X_train_scaled, y_train)}')
print(f'Testing Score: {brfc.score(X_test_scaled, y_test)}')

Training Score: 0.860303805316593
Testing Score: 0.8585294972391747


In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import accuracy_score
brfc = brfc.fit(X_train_scaled, y_train)
predictions = brfc.predict(X_test_scaled)

acc_score = accuracy_score(y_test, predictions)
acc_score

0.8585294972391747

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all our features in X
model.fit(X, y)

score = model.score(X, y)
print(f"R2 Score: {score}")

In [ ]:
predictions = model.predict(X)
# Plot Residuals
plt.scatter(predictions, predictions - y)
plt.hlines(y=0, xmin=predictions.min(), xmax=predictions.max())
plt.show()

In [12]:
# Display the confusion matrix
# YOUR CODE HERE
# cm = confusion_matrix(y_test, predictions)

# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# cm_df

In [11]:
# Print the imbalanced classification report
# YOUR CODE HERE
# from imblearn.metrics import classification_report_imbalanced
# print(classification_report_imbalanced(y_test, predictions))

In [6]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
# importances = brfc.feature_importances_
# importances
# sorted(zip(brfc.feature_importances_, X.columns), reverse=True)

### Easy Ensemble AdaBoost Classifier

In [10]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE
# from imblearn.ensemble import EasyEnsembleClassifier
# easy_classifier = EasyEnsembleClassifier(n_estimators=100, random_state=1)
# easy_classifier.fit(X_train_scaled, y_train)
# predictions2 = easy_classifier.predict(X_test_scaled)
# predictions2


In [7]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
# from sklearn.metrics import accuracy_score

# bal_acc = accuracy_score(y_test, predictions2)
# print(bal_acc)

In [8]:
# Display the confusion matrix
# YOUR CODE HERE
# cm = confusion_matrix(y_test, predictions2)

# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

# cm_df

In [9]:
# Print the imbalanced classification report
# YOUR CODE HERE
# Print the imbalanced classification report
# from imblearn.metrics import classification_report_imbalanced

# print(classification_report_imbalanced(y_test, predictions2))